In [8]:
# la fuente del codigo es: https://github.com/sankit1/cv-tricks.com

#importa la clase conjuntoDeDatos, esta obtiene las imagenes de entrenamiento por cada subcarpeta o clases de imagenes. 
import conjuntoDeDatos
# importa lar libreria de tensorflow.
import tensorflow as tf
import time
from datetime import timedelta
import math
import random
import numpy as np

# Adding Seed so that random initialization is consistent
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)

""" tamanoLotePorIteracion (batch-size) """
tamanoLotePorIteracion = 32

#Preparar la data de entrada
#Existen dos calses, perros y gatos.
clases = ['dogs','cats']
numeroClases = len(clases)

# leendo las imagenes de entrada
# datos de entrenamiento: deberia ser usado el 80% de las imagenes
# datos de validacion: deberia ser usado el 20% de las imagenes, estas imagenes deben quedar fuera del conjunto de datos de entrenamiento, este conjunto de datos es necesario para calcular la exactitud del modelo.
# datos de prueba, este conjunto de datos es utilizado para probar el modelo, despue de haber entrenado el modelo funciona bien, pero cuando las imagenes son convertidas a un tamaño muy reducido el model puede fallar, a este termino se le conoce como sobre sobreajuste(Overfitting), el sobre ajuste puede ser probocado por el fondo de las imagenes.

# tamanoDeDataDeValidacion automaticamente se usara el 20% de todas las imagenes para la validacion.
tamanoDeDataDeValidacion = 0.2

# El mamaño de las imagenes es de 128 pixeles por 128.
tamanoDeImagenes = 128

# el numero de canales representa que la imagenen tiene los tres colores RGB
numeroDeColoresPorImagen = 3

# la ruta de datos de entrenamiento es la carpeta donde se encuentran los conjuntos de imagenes de entrenamiento y validacion
rutaDeDatosDeEntrenamiento='datosDeEntrenamiento'

# Cargar las imagenes de entrenamiento y validacion con sus etiquetas en memoria usando openCV para ser usadas durante el proceso de entrenamiento.
data = conjuntoDeDatos.leerDatosDeEntrenamiento(rutaDeDatosDeEntrenamiento, tamanoDeImagenes, clases, validation_size=tamanoDeDataDeValidacion)

print("Completado el proceso de lecturada de imagenes.")
print("Numero de imagenes en el conjunto de entrenamiento:\t{}".format(len(data.train.labels)))
print("Numero de imagenes en el conjunto de validacion :\t{}".format(len(data.valid.labels)))

Going to read training images
Now going to read dogs files (Index: 0)
Now going to read cats files (Index: 1)
Completado el proceso de lecturada de imagenes.
Numero de imagenes en el conjunto de entrenamiento:	16
Numero de imagenes en el conjunto de validacion :	4


In [9]:
session = tf.Session()
x = tf.placeholder(tf.float32, shape=[None, tamanoDeImagenes,tamanoDeImagenes,numeroDeColoresPorImagen], name='x')

## Etiquetas
tensorDeClases = tf.placeholder(tf.float32, shape=[None, numeroClases], name='tensorDeClases')
tensorDeClasesAplanado = tf.argmax(tensorDeClases, dimension=1)

##Parametros graficos de red
# El parametro mas importante en una red convolucional es el tamaño del filtro de cada neurona.  Por ejemplo si entrada de la neurona es una imagen de 32x32x3, esto significa que la imagen tiene una dimension de 32x32 (alto x ancho) pixeles y contiene 3 colores RGB, entonces el filtro significa que se convertiran las imagenes a 5x5x3, es decir 5x5 (alto x ancho) y 3 representa el numero de colores RGB.

#Configuracion de la primera capa oculta, el tamaño del filtro es 3 y el numero de filtros es 32
#filter_size_conv1
#num_filters_conv1
tamanoFiltroCapaConvolucional1 = 3
numeroFiltrosCapaConvolucional1 = 32

tamanoFiltroCapaConvolucional2 = 3
numeroFiltrosCapaConvolucional2 = 32

tamanoFiltroCapaConvolucional3 = 3
numeroFiltrosCapaConvolucional3 = 64

tamanoDeCapaTotalmenteConectada = 128

In [10]:
# crear pesos
def crearPesos(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.05))

#crear sesgos
def crearSesgos(size):
    return tf.Variable(tf.constant(0.05, shape=[size]))

#funcion para crear un capa convolucional.
def crearCapaConvolucional(input,num_input_channels, conv_filter_size,num_filters):

    ## We shall define the weights that will be trained using create_weights function.
    ## Debemos definir los pesos que seran entrenables usango la funcion crearPesos
    pesos = crearPesos(shape=[conv_filter_size, conv_filter_size, num_input_channels, num_filters])

    ## We create biases using the create_biases function. These are also trained.
    ## creamos los sesgos usanto la funcion crearSesgos. Estos son tambien entrenados
    sesgos = crearSesgos(num_filters)

    ## Creando la capa convolucional
    capa = tf.nn.conv2d(input=input,
                     filter=pesos,
                     strides=[1, 1, 1, 1],
                     padding='SAME')

    #Agregar los sesgos a la capa.
    capa += sesgos

    ## We shall be using max-pooling. see more information on https://www.quora.com/What-is-max-pooling-in-convolutional-neural-networks
    # Estaremos usando el metodo cnn.max-pooling de tensorflow. El algoritmo realiza el maximo agrupamiento en la entrada
    ##Los parametros de cnn.max_pool son:
    # value =  Un tendor de 4 dimension con formato especificado por data_format.
    # ksize =  Un tensor de tipo entero (int) de 1 dimension con 4 elementos.  
    ##         Representan el tama;o de la ventana para cada dimension del tensor de entrada.
    # strides= Strides significa pasos. Es un tensor de tipo entero (int) de 1 dimension con 4 elementos.
    ##         El paso de la ventana deslizante para cada dimension del tensor de entrada.
    # padding= Es el relleno.  Es un texto, ya sea SAME o VALID.   Existe un algoritmo para el relleno (padding)
    # ver documentacion de tf.cnn.max_pooling
    capa = tf.nn.max_pool(value=capa,
                            ksize=[1, 2, 2, 1],
                            strides=[1, 2, 2, 1],
                            padding='SAME')

    ## Output of pooling is fed to Relu which is the activation function for us.
    capa = tf.nn.relu(capa)

    # retornar un tensor con el formato especificado en data_format.  Es un tensor resultante con el maximo de agrupamiento.
    return capa


In [11]:

def crearCapaAplanada(capaNoAplanada):
    #We know that the shape of the layer will be [batch_size img_size img_size num_channels]
    #sabemos que la forma de una capa podria ser [tamanoPaquete tamanoImagene tamanoImagen numeroCanales]

    # pero conseguiremos esto desde una capa previa
    # But let's get it from the previous layer.
    formaDeLaCapa = capaNoAplanada.get_shape()

    ## El numero de caracteristica podria ser altoImagen * anchoImagen * numeroCanales.  
    ## Pero nosotros deberiamos calcularlo esto en lugar de codificarlo con a la fuerza o forzado.
    numeroDeCaracteristicas = formaDeLaCapa[1:4].num_elements()

    ## ahora, aplanaremos la capa, asi que tendremos que remodelar (reshape) la cantidad de caracteristicas (num_features)
    capaAplanada = tf.reshape(capaNoAplanada, [-1, numeroDeCaracteristicas])

    #retornar la capa aplanda.
    return capaAplanada

def crearCapaTotalmenteConectada(input,num_inputs,num_outputs,use_relu=True):
    #vamos a definir los pesos y sesgos entrenables
    pesos = crearPesos(shape=[num_inputs, num_outputs])
    sesgos = crearSesgos(num_outputs)

    # Fully connected layer takes input x and produces wx+b.Since, these are matrices, we use matmul function in Tensorflow
    capaTotalmenteConectada = tf.matmul(input, pesos) + sesgos
    if use_relu:
        capaTotalmenteConectada = tf.nn.relu(capaTotalmenteConectada)

    return capaTotalmenteConectada

In [12]:
capaConvolucional1 = crearCapaConvolucional(input=x,
               num_input_channels=numeroDeColoresPorImagen,
               conv_filter_size=tamanoFiltroCapaConvolucional1,
               num_filters=numeroFiltrosCapaConvolucional1)

capaConvolucional2 = crearCapaConvolucional(input=capaConvolucional1,
               num_input_channels=numeroFiltrosCapaConvolucional1,
               conv_filter_size=tamanoFiltroCapaConvolucional2,
               num_filters=numeroFiltrosCapaConvolucional2)

capaConvolucional3= crearCapaConvolucional(input=capaConvolucional1,
               num_input_channels=numeroFiltrosCapaConvolucional2,
               conv_filter_size=tamanoFiltroCapaConvolucional3,
               num_filters=numeroFiltrosCapaConvolucional3)

capaPlana = crearCapaAplanada(capaConvolucional3)

capaTotalmenteConectada1 = crearCapaTotalmenteConectada(input=capaPlana,
                     num_inputs=capaPlana.get_shape()[1:4].num_elements(),
                     num_outputs=tamanoDeCapaTotalmenteConectada,
                     use_relu=True)

print("crear capa totalmente concetada 1, tamanoDeCapaTotalmenteConectada= "+str(tamanoDeCapaTotalmenteConectada)+", numeroClases= "+str(numeroClases))

capaTotalmenteConectada2 = crearCapaTotalmenteConectada(input=capaTotalmenteConectada1,
                     num_inputs=tamanoDeCapaTotalmenteConectada,
                     num_outputs=numeroClases,
                     use_relu=False)

print("crear capa totalmente concetada 2, tamanoDeCapaTotalmenteConectada= "+str(tamanoDeCapaTotalmenteConectada)+", numeroClases= "+str(numeroClases))

#Softmax: Calcula las activaciones softmax, los argumentos de la funcion son:
## logits: es un tenson no vacio.  Puede ser uno de los siguientes tipos "half, float32, float64"
## dim: la dimension softmax puede ser realidad en: el valor default es -1, el cual indica la ultima dimension.
## name: es opcional, representa el nombre de la operacion.

## retorna: un tensor, tiene el mismo tipo y forma que logits.

# la variable ResultadoDeProbabilidadPorClase guarda los valores de la ultima capa (capaTotalmenteConectada2) y obtiene por medio de la funcion softmax los valores de probabilidad en un rango de 0-1
prediccionDeProbabilidadPorClase = tf.nn.softmax(capaTotalmenteConectada2,name='prediccionDeProbabilidadPorClase')

# la variable ResultadoDeProbabilidadPorClase guarda la probabilidad de cada clase aplanda es decir en un vector.
prediccionDeProbabilidadPorClaseAplanada = tf.argmax(prediccionDeProbabilidadPorClase, dimension=1)

# tf.Session es una clase para correr operaciones de tensorflow
# Un objetivo de una sesion es encapsular el entorno en el cual los objectos (operaciones) son ejecutados y los objectos (Tensores) son evaluados.

#inicializar las variables.
session.run(tf.global_variables_initializer())

#Entropia cruzada:
entropiaCruzada = tf.nn.softmax_cross_entropy_with_logits(logits=capaTotalmenteConectada2,labels=tensorDeClases)
perdidaTotalDeErrorEnClasificacion = tf.reduce_mean(entropiaCruzada)
optimizador = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(perdidaTotalDeErrorEnClasificacion)
correccionDePrediccion = tf.equal(prediccionDeProbabilidadPorClaseAplanada, tensorDeClasesAplanado)
exactitudDelModelo = tf.reduce_mean(tf.cast(correccionDePrediccion, tf.float32))

#inicializar las variables.
session.run(tf.global_variables_initializer())

crear capa totalmente concetada 1, tamanoDeCapaTotalmenteConectada= 128, numeroClases= 2
crear capa totalmente concetada 2, tamanoDeCapaTotalmenteConectada= 128, numeroClases= 2


<font color='red'>**Jupyter Kernel terminated:**</font> This might be caused by running out of memory or hitting a bug in some library (e.g., forking too many processes, trying to access invalid memory, etc.). Consider restarting or upgrading your project or running the relevant code directly in a terminal to track down the cause, as [explained here](https://github.com/sagemathinc/cocalc/wiki/KernelTerminated).

In [0]:
def mostrarProgreso(epoca, feed_dict_train, feed_dict_validate, perdidaDeValidacion):
    exactitudDeEntrenamiento = session.run(exactitudDelModelo, feed_dict=feed_dict_train)
    exactitudDeValidacion = session.run(exactitudDelModelo, feed_dict=feed_dict_validate)
    msg = "Training epoca {0} --- Exactitud del entranamiento: {1:>6.1%}, Exactitud de validacion: {2:>6.1%},  perida de validacion: {3:.3f}"
    print(msg.format(epoca + 1, exactitudDeEntrenamiento, exactitudDeValidacion, perdidaDeValidacion))

recuentoIteraciones = 0

""" batch-size (tamaño del lote por iteracion), cuando se entrena una red no se  alimenta con todo el conjunto de datos, se hace por medio de iteracion y cada iteracion tiene un tamaño, frecuentemente es 16 o 32"""

tamanoLotePorIteracion = 4
saver = tf.train.Saver()

def entrenarModelo(totalIteraciones):
    print("---------------- FUNCION DE ENTRENAMIENTO  ------------------")
    print("numero de imagenes para entrenamiento "+str(data.train.num_examples))

    global recuentoIteraciones
    for iteracion in range(recuentoIteraciones,recuentoIteraciones + totalIteraciones):
        print("-- iteracion # "+str(iteracion+1))
        x_batch, y_true_batch, _, cls_batch = data.train.siguienteLote(tamanoLotePorIteracion)
        x_valid_batch, y_valid_batch, _, valid_cls_batch = data.valid.siguienteLote(tamanoLotePorIteracion)

        feed_dict_tr = {x: x_batch,tensorDeClases: y_true_batch}
        feed_dict_val = {x: x_valid_batch,tensorDeClases: y_valid_batch}
        session.run(optimizador, feed_dict=feed_dict_tr)

        if iteracion % int(data.train.num_examples/tamanoLotePorIteracion) == 0:
            print("mostrar epoca "+str(int(data.train.num_examples/tamanoLotePorIteracion)))
            perdidadDeValidacion = session.run(perdidaTotalDeErrorEnClasificacion, feed_dict=feed_dict_val)

            """ epoca (epoch) una epoca contiene un conjunto de iteracion que alimentan el modelo """
            epoca = int(iteracion / int(data.train.num_examples/tamanoLotePorIteracion))

            mostrarProgreso(epoca, feed_dict_tr, feed_dict_val, perdidadDeValidacion)
            """saver.save(session, 'dogs-cats-model')"""

    recuentoIteraciones += totalIteraciones

entrenarModelo(totalIteraciones=6)
print("end")

<font color='red'>**Jupyter Kernel terminated:**</font> This might be caused by running out of memory or hitting a bug in some library (e.g., forking too many processes, trying to access invalid memory, etc.). Consider restarting or upgrading your project or running the relevant code directly in a terminal to track down the cause, as [explained here](https://github.com/sagemathinc/cocalc/wiki/KernelTerminated).